# Google Colab [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedrocamargo/road_analytics/blob/master/notebooks/GoogleColabExample.ipynb)

If you would like to run this notebook yourself - simply make a copy of this notebook in Google Colab by clicking the badge above.

This will create a copy under your user account which you can modify and experiment with. 

In [ ]:
import sys
!{sys.executable} -m pip install aequilibrae geopandas  matplotlib contextily tabulate spatialite
!apt-get install libsqlite3-mod-spatialite
!apt-get install -y libspatialite-dev
!ln -s /usr/lib/x86_64-linux-gnu/mod_spatialite.so /usr/lib/x86_64-linux-gnu/mod_spatialite
!{sys.executable} -m pip install spatialite pysqlite3-binary

# Download an example aequilibrae project

This project is currently hosted on Outerloop infrastructure - but we'd hope to have this hosted in the Bank data lake when that is operational

In [ ]:
from urllib.request import urlretrieve

url = "https://s3.amazonaws.com/outerloop.io/Armenia.zip"
urlretrieve(url, "./Armenia.zip")
!unzip Armenia.zip

### Load the downloaded project

In [ ]:
from aequilibrae import Project
project = Project()
project.load("./Armenia")

In [ ]:
plot_network("Armenia", project)

In [ ]:
links = project.network.links.data
links = links[links.modes.str.contains('c')]

basic_stats(links)
bridge_stats(links)

# Function definitions

During development functions can be defined in the notebooks that they are used in.

When they are more stable - they can be pulled into a pip installable library and used in any notebook (see the Aequilbrae pip installed above)


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import matplotlib.lines as mlines
from aequilibrae import Project


import time
import re
import requests
from aequilibrae.project.network.osm_utils.osm_params import http_headers, memory
from shapely.geometry import MultiPolygon, Point, LineString
from shapely.ops import polygonize
from os.path import dirname, join
import pandas as pd
import shapely.wkb
import sqlite3


def plot_network(country_name:str, project):
    
    links = project.network.links.data
    links = links[links.modes.str.contains('c')]

    country_border = get_country_borders(project)

    geo_links = gpd.GeoDataFrame(links, geometry='geometry', crs=4326)

    geo_borders = gpd.GeoDataFrame(pd.DataFrame(country_border, columns=['geometry']),
                                   geometry='geometry', crs=4326)

    line_dict = {'motorway':3.3, 'trunk':2.5, 'primary':2.5, 'secondary':1.5}

    road_palette = {'motorway':"#8E1916", 'trunk':"#013a63", 
                    'primary':"#dd8024", 'secondary':"#40916c"}

    color_list = ["#52b788","#74c69d","#95d5b2","#b7e4c7","#89c2d9","#61a5c2","#468faf",
                  "#2a6f97", "#B8191B", "#d5ca7a", "#efb337"]

    print('1.')
    for i in geo_links.link_type.unique().tolist(): #adicionar estrutura de controle
        if i not in line_dict:
            line_dict[i] = 0.9
            try:
                road_palette[i] = color_list.pop()
            except ValueError:
                road_palette[i] = 'darkgray'
        pass
    
    # Plot data
    fig, ax = plt.subplots(figsize=(20, 20))

    print('1.')
    geo_borders.plot(figsize=(20,20), color='gainsboro', alpha=0.8, ax=ax, edgecolor='black', label='Country Borders', marker='s')

    for ctype, data in geo_links.groupby('link_type'):
        color = road_palette[ctype]
        label = ctype    
        data.plot(color=color,
                  ax=ax,
                  linewidth=line_dict[ctype], 
                  label=label,
                  marker='_')

    print('1.')
    cx.add_basemap(ax, crs=4326, source=cx.providers.Stamen.TonerLite) 
    
    line = mlines.Line2D([], [], color='gainsboro', marker='s', markersize=14, label='Country Border', ls='None',
                         markeredgecolor='black', alpha=0.8)

    handles, _ = ax.get_legend_handles_labels()

    ax.legend(loc='best', fontsize=12, title=f"OSM Network Links", framealpha=1, title_fontsize=16, handles=[*handles, line])

    ax.set_axis_off()

    plt.show()


def get_country_borders(project: Project):
    country_wkb = project.conn.execute('Select asBinary(geometry) from country_borders').fetchone()[0]
    polyg =  MultiPolygon(shapely.wkb.loads(country_wkb))
    
        
    return polyg

In [ ]:
from tabulate import tabulate
import pandas as pd

def basic_stats(links):
    
    vehicle_links = links[(links["modes"].str.contains("c")) & (links["modes"].str.contains("t"))]
    
    print("----- Links -----")
    print("")
    bridge_links = len(vehicle_links[vehicle_links['bridge'].notna()])
    toll_links = len(vehicle_links[vehicle_links['toll'].notna()])
    tunnel_links = len(vehicle_links[vehicle_links['tunnel'].notna()])
    total_links = len(vehicle_links)
    
    table = [[total_links - bridge_links - toll_links - tunnel_links, bridge_links, toll_links, tunnel_links, total_links]]
    
    print(tabulate(table, headers=["General", "Bridge", "Toll", "Tunnel", "Total"], tablefmt="github", floatfmt=".0f"))
    print("")

    print("----- Total km -----")
    print("")
    bridge_dist = vehicle_links[vehicle_links['bridge'].notna()]['distance'].sum()/1000
    toll_dist = vehicle_links[vehicle_links['toll'].notna()]['distance'].sum()/1000
    tunnel_dist = vehicle_links[vehicle_links['tunnel'].notna()]['distance'].sum()/1000
    total_dist = vehicle_links.distance.sum()/1000
    
    table = [[total_dist - bridge_dist - toll_dist - tunnel_dist,
              bridge_dist, toll_dist, tunnel_dist, total_dist]]
    print(tabulate(table, headers=["General", "Bridge", "Toll", "Tunnel", "Total"], tablefmt="github", floatfmt=".2f"))
    print("")
    
    print("----- Link Type -----")
    print("")
    df = pd.DataFrame.from_dict(dict(vehicle_links.groupby('link_type').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', vehicle_links.groupby('link_type').count()['distance'])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".2f"), 
                   headers=["Link Type", "Links", 'km']))
    print("")
    
    print("----- Pavement Surfaces -----")
    print("")
    vehicle_links.loc[vehicle_links['surface'].isna(), 'surface'] = 'unclassified'
    df = pd.DataFrame.from_dict(dict(vehicle_links.groupby('surface').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', vehicle_links.groupby('surface').count()['distance'])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".2f"), 
                   headers=["Surface Type", "Links", 'km']))
    print("")

In [ ]:
from tabulate import tabulate
import pandas as pd
import numpy as np

def bridge_stats(links):
    
    vehicle_links = links[(links["modes"].str.contains("c")) & (links["modes"].str.contains("t"))]
    bridge_links = vehicle_links[vehicle_links["bridge"].notna()]
    
    print("----- Bridge - Overall Stats -----")
    print(" ")
    table = [["Bridge", len(bridge_links), 
              np.around(bridge_links.sum(numeric_only=True)['distance'], decimals=2)],
             ["Total", len(vehicle_links), 
              np.around(vehicle_links.sum(numeric_only=True)['distance'], decimals=2)]]

    print(tabulate(table, headers=["", "Links", "km"], tablefmt='github', floatfmt=".2f"))
    print(" ")
    if len(bridge_links[bridge_links['toll'].notna()]) == 0:
        print(f"There are no toll bridges in the search area.")
        print("")
    else: 
        print(f"----- Toll Bridge -----")
        print(" ")
        bridge_bridges = bridge_links[bridge_links['toll'] == 'yes']
        table = [[len(bridge_bridges),
                  bridge_bridges.sum(numeric_only=True)['distance']/1000]]

        print(tabulate(table, headers=['links', 'km'], tablefmt='github', floatfmt=".2f"))
        print("")
    
    print("----- Bridge - Link Type -----")
    print("")
    df = pd.DataFrame.from_dict(dict(bridge_links.groupby('link_type').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', bridge_links.groupby('link_type').count()['distance'])
    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".2f"), 
                   headers=["Link Type", "Links", 'km']))
    print("")
    
    print("----- Bridge - Pavement Surface -----")
    print("")
    bridge_links.loc[bridge_links['surface'].isna(), 'surface'] = 'unclassified'
    df = pd.DataFrame.from_dict(dict(bridge_links.groupby('surface').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', bridge_links.groupby('surface').count()['distance'])
    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".2f"), 
                   headers=["Surface Type", "Links", 'km']))
    print("")